In [1]:
class predictions():
    
    def __init__(self):
        self.vectors = vectorizer()
        
    #get the vector for predictions
    def get_predict_vector(self,string):
        return self.vectors.reVectorize(self.vectors.dialogue_act_features(string))
    
    def get_val_for_index(self,array):
        for i in array:
            www = self.index_to_val[i]
        return www
    
    def predict(self,string):
        vector = self.get_predict_vector(string)
        res = self.clf.predict(vector)
        return self.get_val_for_index(res)
    
    def learn(self,x):
        temp = self.vectors.train_gen(x)
        X = self.vectors.vectorize(temp['x'])
        Y,self.index_to_val = dict_builder(temp['y'])
        self.clf = LinearSVC().fit(X, Y)
    def learn_from_db(self,x,xy_list):
        temp = self.vectors.train_gen(x)
        for i in xy_list['x']:
            temp['x'].append(i)
        for i in xy_list['y']:
            temp['y'].append(i)
        X = self.vectors.vectorize(temp['x'])
        Y,self.index_to_val = dict_builder(temp['y'])
        self.clf = LinearSVC().fit(X, Y)

In [2]:
#NLTK IMPORTS
import nltk
from nltk.corpus import stopwords


#PATTERNS
from pattern.en import parsetree
from pattern.en import tenses, PAST, PL

#IMPORTING STUFF
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


#SCIKIT-LEARN IMPORTS
from sklearn.svm import LinearSVC


class vectorizer():
	def __init__(self):
		self.stopwords = stopwords.words('english')
		self.vec = DictVectorizer()
        
	def tenseses(self,strs):
		if 'present' in str(tenses(strs)):
		    ten = 'PRESENT'
		elif 'past' in tenses(strs):
		    ten = "PAST"
		else:
		    ten = "FUTURE"
		return ten

	def dialogue_act_features(self,post):
		stop = nltk.word_tokenize(post)
		post = []
		for i in stop:
		    if i not in self.stopwords:
			post.append(i)
		    else:
			pass
			#regect_list.append(i)
		posts = ""
		for i in post:
		    posts += i
		    posts += " "
		processed = parsetree(posts, relations=True, lemmata=True)
		features = {}
		for sents in processed:
		    x = sents
		    for i in x.chunks:
			j = i.pos
			if j == "VP":
			    tense = self.tenseses(i.string)
			else:
			    tense = ""
			h = i.words
			for words in h:
			    apd_str =  str(words.lemma) +"-"+ str(words.pos)
			    #+ "-" +tense
			    #if words.pos[:1] == "NN" or words.pos[:1] == "VP":
			    features['word({})'.format(str(words.lemma))] =  str(words.pos)
			#pos['features({})'.format(j)] = True
		    return features
	def vectorize(self,feat_list):
		pos_vectorized = self.vec.fit_transform(feat_list)
		return pos_vectorized
	def reVectorize(self,feat_list):
		pos_vectorized = self.vec.transform(feat_list)
		return pos_vectorized
	def train_gen(self,x):
	    iterations = []
	    for i in x:
		#print
		m1 = i[0]
		m1.reverse()
		x1 = i[1]
		x1.reverse()
		while True:
		    if len(x1) == 0 and len(m1) == 0:
			break
		    temp = []
		    temp2 = []
		    while True:
			try:
			    temp.append(m1.pop())
			    #print temp[-1]
			    if temp[1] == 0:
			        break
			except Exception as e:
			    break
		    while True:
			try:
			    temp2.append(x1.pop())
			    #print temp2[-1]
			    if temp2[-1][1] == 0:
			        break
			except Exception as e:
			    break
		    #print len(x1)
		    me_str = ""
		    m_str = "" 
		    for i in temp:
			me_str += " "
			me_str += i[0]
		    for i in temp2:
			m_str += i[0]
		    iterations.append((self.dialogue_act_features(me_str),m_str))
	    x = []
	    y = []
	    for i in iterations:
	    	  x.append(i[0])
	    	  y.append(i[1])
	    return_dictionary_buffer = {}
	    return_dictionary_buffer['x'] = x
	    return_dictionary_buffer['y'] = y  
	    return return_dictionary_buffer

def dict_builder(y):
    count = 0
    indexDict = {}
    reverseDict = {}
    indexList = []
    for i in y:
        try:
            indexList.append(reverseDict[i])
        except Exception as e:
            indexList.append(count)
            reverseDict[i] = count
            count += 1
    for i in reverseDict:
        indexDict[reverseDict[i]] = i
    return (indexList,indexDict) 


In [64]:
class predictions():
    
    def __init__(self):
        self.vectors = vectorizer()
        
    #get the vector for predictions
    def get_predict_vector(self,string):
        return self.vectors.reVectorize(self.vectors.dialogue_act_features(string))
    
    def get_val_for_index(self,array):
        for i in array:
            www = self.index_to_val[i]
        return www
    
    def predict(self,string):
        vector = self.get_predict_vector(string)
        res = self.clf.predict(vector)
        return self.get_val_for_index(res)
    
    def learn(self,x):
        temp = self.vectors.train_gen(x)
        print temp
        X = self.vectors.vectorize(temp['x'])
        Y,self.index_to_val = dict_builder(temp['y'])
        self.clf = LinearSVC().fit(X, Y)
    def learn_from_db(self,x,xy_list):
        temp = self.vectors.train_gen(x)
        for i in xy_list['x']:
            temp['x'].append(self.vectors.dialogue_act_features(i))
        for i in xy_list['y']:
            temp['y'].append(i)
        print temp
        X = self.vectors.vectorize(temp['x'])
        Y,self.index_to_val = dict_builder(temp['y'])
        self.clf = LinearSVC().fit(X, Y)

In [65]:
vector = predictions()

In [66]:
ter = {}
ter['x'] = ['hey','howdy']
ter['y'] =  ['hey','hey']
x = [[[[u'save', 0]], []], [[[u'hey', 0]], [[u'hey', 0]]], [[], []], [[[u'hey', 0], [u'I need to fix my hard disk', 0], [u'thanks', 0], [u'bye', 0]], [[u'hey', 0], [u'computer', 0], [u'thankyou', 0], [u'bye', 0]]], [[], []], [[[u'hye', 0], [u'I need to change the color of my room', 0], [u'thankyou', 0], [u'bye', 0]], [[u'hey', 0], [u'paint', 0], [u'thankyou', 0], [u'bye', 0]]], [[[u'hello', 0], [u'I need to hire a painter', 0], [u'thank you so much', 0], [u'bye', 0]], [[u'hey', 0], [u'paint', 0], [u'thankyou', 0], [u'bye', 0]]], [[[u'save', 0]], []], [[[u'hey', 1], [u'I need to find a yoga guru.', 0], [u'how much does he charge?', 0], [u'okay', 0], [u'yes', 0], [u'see you later', 0]], [[u'hey', 0], [u'yoga', 0], [u'price', 0], [u'confirm', 0], [u'positive', 0], [u'bye', 0]]], [[[u'hello', 0], [u'I need someone to clean my house.', 0], [u'How much do I have to pay', 0], [u'that is okay', 0], [u'bye', 0]], [[u'hey', 0], [u'clean', 0], [u'price', 0], [u'confirm', 0], [u'bye', 0]]], [[], []], [[[u'I think my air-conditioner is not working', 0]], [[u'cools', 0]]], [[], []], [[[u'I need to paint of my room', 0], [u'the television in my house is not working', 0], [u'my home theater is giving some problem', 0], [u'my water purifier is not working', 0], [u'I need to make some furniture.', 0], [u'chair', 0], [u'cupboard', 0], [u'wood', 0], [u'table', 0], [u'stand', 0], [u'plug not working in house', 0], [u'My electric sockets are sparking', 0]], [[u'paint', 0], [u'house_app', 0], [u'house_app', 0], [u'house_app', 0], [u'carpy', 0], [u'carpy', 0], [u'carpy', 0], [u'carpy', 0], [u'carpy', 0], [u'carpy', 0], [u'electric', 0], [u'electric', 0]]], [[], []], [[[u'hey', 0], [u'fix my pipes', 0], [u'pipes are leaking', 0], [u'install pipeline', 0], [u'drainage blocked', 0]], [[u'hey', 0], [u'plumb', 0], [u'plumb', 0], [u'plumb', 0], [u'plumb', 0]]], [[], []], [[[u'keyboard mouse harddisk hard disk screen display webcam', 0], [u'hard disk is not working', 0], [u'My laptop is over heating', 0], [u'my keyboard is not working', 0], [u'my mouse is not working', 0], [u'my computer is not turning on', 0]], [[u'computer', 0], [u'computer', 0], [u'computer', 0], [u'computer', 0], [u'computer', 0], [u'computer', 0]]], [[], []], [[[u'My hard disk is broken.', 0]], [[u'computer', 0]]], [[], []]]


In [67]:

vector.learn(x)

x = [[[[u'save', 0]], []], [[[u'hey', 0]], [[u'hey', 0]]], [[], []], [[[u'hey', 0], [u'I need to fix my hard disk', 0], [u'thanks', 0], [u'bye', 0]], [[u'hey', 0], [u'computer', 0], [u'thankyou', 0], [u'bye', 0]]], [[], []], [[[u'hye', 0], [u'I need to change the color of my room', 0], [u'thankyou', 0], [u'bye', 0]], [[u'hey', 0], [u'paint', 0], [u'thankyou', 0], [u'bye', 0]]], [[[u'hello', 0], [u'I need to hire a painter', 0], [u'thank you so much', 0], [u'bye', 0]], [[u'hey', 0], [u'paint', 0], [u'thankyou', 0], [u'bye', 0]]], [[[u'save', 0]], []], [[[u'hey', 1], [u'I need to find a yoga guru.', 0], [u'how much does he charge?', 0], [u'okay', 0], [u'yes', 0], [u'see you later', 0]], [[u'hey', 0], [u'yoga', 0], [u'price', 0], [u'confirm', 0], [u'positive', 0], [u'bye', 0]]], [[[u'hello', 0], [u'I need someone to clean my house.', 0], [u'How much do I have to pay', 0], [u'that is okay', 0], [u'bye', 0]], [[u'hey', 0], [u'clean', 0], [u'price', 0], [u'confirm', 0], [u'bye', 0]]], [[], []], [[[u'I think my air-conditioner is not working', 0]], [[u'cools', 0]]], [[], []], [[[u'I need to paint of my room', 0], [u'the television in my house is not working', 0], [u'my home theater is giving some problem', 0], [u'my water purifier is not working', 0], [u'I need to make some furniture.', 0], [u'chair', 0], [u'cupboard', 0], [u'wood', 0], [u'table', 0], [u'stand', 0], [u'plug not working in house', 0], [u'My electric sockets are sparking', 0]], [[u'paint', 0], [u'house_app', 0], [u'house_app', 0], [u'house_app', 0], [u'carpy', 0], [u'carpy', 0], [u'carpy', 0], [u'carpy', 0], [u'carpy', 0], [u'carpy', 0], [u'electric', 0], [u'electric', 0]]], [[], []], [[[u'hey', 0], [u'fix my pipes', 0], [u'pipes are leaking', 0], [u'install pipeline', 0], [u'drainage blocked', 0]], [[u'hey', 0], [u'plumb', 0], [u'plumb', 0], [u'plumb', 0], [u'plumb', 0]]], [[], []], [[[u'keyboard mouse harddisk hard disk screen display webcam', 0], [u'hard disk is not working', 0], [u'My laptop is over heating', 0], [u'my keyboard is not working', 0], [u'my mouse is not working', 0], [u'my computer is not turning on', 0]], [[u'computer', 0], [u'computer', 0], [u'computer', 0], [u'computer', 0], [u'computer', 0], [u'computer', 0]]], [[], []], [[[u'My hard disk is broken.', 0]], [[u'computer', 0]]], [[], []]]

vector.learn_from_db(x,ter)

{'y': ['', u'hey', u'hey', u'computer', u'thankyou', u'bye', u'hey', u'paint', u'thankyou', u'bye', u'hey', u'paint', u'thankyou', u'bye', '', u'hey', u'yoga', u'price', u'confirm', u'positive', u'bye', u'hey', u'clean', u'price', u'confirm', u'bye', u'cools', u'paint', u'house_app', u'house_app', u'house_app', u'carpy', u'carpy', u'carpy', u'carpy', u'carpy', u'carpy', u'electric', u'electric', u'hey', u'plumb', u'plumb', u'plumb', u'plumb', u'computer', u'computer', u'computer', u'computer', u'computer', u'computer', u'computer'], 'x': [{'word(save)': 'VB'}, {}, {}, {'word(disk)': 'NN', 'word(fix)': 'VB', 'word(i)': 'PRP', 'word(hard)': 'JJ', 'word(need)': 'VBP'}, {'word(thank)': 'NNS'}, {'word(bye)': 'VB'}, {'word(hye)': 'NN'}, {'word(room)': 'NN', 'word(color)': 'NN', 'word(i)': 'PRP', 'word(change)': 'NN', 'word(need)': 'VBP'}, {'word(thankyou)': 'PRP'}, {'word(bye)': 'VB'}, {}, {'word(hire)': 'VB', 'word(painter)': 'NN', 'word(i)': 'PRP', 'word(need)': 'MD'}, {'word(thank)': 'NN'

In [68]:
temp = {'y': ['', u'hey', u'hey', u'computer', u'thankyou', u'bye', u'hey', u'paint', u'thankyou', u'bye', u'hey', u'paint', u'thankyou', u'bye', '', u'hey', u'yoga', u'price', u'confirm', u'positive', u'bye', u'hey', u'clean', u'price', u'confirm', u'bye', u'cools', u'paint', u'house_app', u'house_app', u'house_app', u'carpy', u'carpy', u'carpy', u'carpy', u'carpy', u'carpy', u'electric', u'electric', u'hey', u'plumb', u'plumb', u'plumb', u'plumb', u'computer', u'computer', u'computer', u'computer', u'computer', u'computer', u'computer', 'hey', 'hey'], 'x': [{'word(save)': 'VB'}, {}, {}, {'word(disk)': 'NN', 'word(fix)': 'VB', 'word(i)': 'PRP', 'word(hard)': 'JJ', 'word(need)': 'VBP'}, {'word(thank)': 'NNS'}, {'word(bye)': 'VB'}, {'word(hye)': 'NN'}, {'word(room)': 'NN', 'word(color)': 'NN', 'word(i)': 'PRP', 'word(change)': 'NN', 'word(need)': 'VBP'}, {'word(thankyou)': 'PRP'}, {'word(bye)': 'VB'}, {}, {'word(hire)': 'VB', 'word(painter)': 'NN', 'word(i)': 'PRP', 'word(need)': 'MD'}, {'word(thank)': 'NN', 'word(much)': 'RB'}, {'word(bye)': 'VB'}, {'word(save)': 'VB'}, {}, {'word(find)': 'VB', 'word(guru)': 'NN', 'word(yoga)': 'NN', 'word(i)': 'PRP', 'word(need)': 'VBP'}, {'word(charge)': 'NN', 'word(much)': 'JJ'}, {'word(okay)': 'JJ'}, {'word(yes)': 'RB'}, {'word(later)': 'RB', 'word(see)': 'VB'}, {}, {'word(someone)': 'NN', 'word(clean)': 'JJ', 'word(i)': 'PRP', 'word(house)': 'NN', 'word(need)': 'VBP'}, {'word(pay)': 'VBP', 'word(i)': 'PRP', 'word(much)': 'JJ'}, {'word(okay)': 'JJ'}, {'word(bye)': 'VB'}, {'word(think)': 'VBP', 'word(i)': 'PRP', 'word(work)': 'VBG', 'word(air-conditioner)': 'NN'}, {'word(room)': 'NN', 'word(i)': 'PRP', 'word(paint)': 'NN', 'word(need)': 'VBP'}, {'word(television)': 'NN', 'word(house)': 'NN', 'word(working)': 'NN'}, {'word(give)': 'VBG', 'word(problem)': 'NN', 'word(home)': 'NN', 'word(theater)': 'NN'}, {'word(purifier)': 'NN', 'word(water)': 'NN', 'word(working)': 'NN'}, {'word(make)': 'VB', 'word(furniture)': 'NN', 'word(i)': 'PRP', 'word(need)': 'MD'}, {'word(chair)': 'NN'}, {'word(cupboard)': 'NN'}, {'word(wood)': 'NN'}, {'word(table)': 'NN'}, {'word(stand)': 'VBP'}, {'word(working)': 'NN', 'word(house)': 'NN', 'word(plug)': 'NN'}, {'word(electric)': 'JJ', 'word(my)': 'PRP$', 'word(socket)': 'NNS', 'word(spark)': 'VBG'}, {}, {'word(fix)': 'JJ', 'word(pipe)': 'NNS'}, {'word(leak)': 'VBG', 'word(pipe)': 'NNS'}, {'word(pipeline)': 'NN', 'word(install)': 'VB'}, {'word(drainage)': 'NN', 'word(block)': 'VBN'}, {'word(keyboard)': 'NN', 'word(hard)': 'JJ', 'word(screen)': 'NN', 'word(display)': 'NN', 'word(mouse)': 'NN', 'word(harddisk)': 'NN', 'word(disk)': 'NN', 'word(webcam)': 'VBD'}, {'word(work)': 'VBG', 'word(disk)': 'NN', 'word(hard)': 'JJ'}, {'word(laptop)': 'JJ', 'word(heating)': 'NN', 'word(my)': 'PRP$'}, {'word(keyboard)': 'NN', 'word(working)': 'NN'}, {'word(mouse)': 'NN', 'word(working)': 'NN'}, {'word(turn)': 'VBG', 'word(computer)': 'NN'}, {'word(my)': 'PRP$', 'word(disk)': 'NN', 'word(hard)': 'JJ', 'word(break)': 'VBN'}, 'hey', 'howdy']}

In [72]:
print temp['y'][-10:-1]

[u'plumb', u'computer', u'computer', u'computer', u'computer', u'computer', u'computer', u'computer', 'hey']
